In [3]:
# 연습문제 9-6.11번가 베스트 아이템 목록 추출하기

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import urllib.request
import urllib

import win32com.client as win32   #pywin32 설치후 동작
import win32api  #pypiwin32 설치후 동작 ,파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                 #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("연습문제 9-6. 11번가 사이트의 Best Seller 상품 정보 추출하기")
print("=" *80)
          
query_txt='11번가'
query_url='http://www.11st.co.kr/html/bestSellerMain.html'

cnt = int(input('   1.크롤링 할 건수는 몇건입니까?(1 - 100 사이값 입력): '))
real_cnt = math.ceil(cnt / 20)

f_dir = input("   2.파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")
print("\n")

if cnt > 30 :
      print("   요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~")
else :
      print("   요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

img_dir = f_dir+s+'-'+query_txt+"\\images"

os.makedirs(img_dir)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'
s_time = time.time( )

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)


driver.get(query_url)
time.sleep(2)

#Step 5. 20건 출력되어 있는 현재 페이지 리뷰와 점수 등 내용 수집

def scroll_down(driver):
      driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
      time.sleep(1)

scroll_down(driver)

ranking2=[]
title2=[]
price2=[]
seller_2=[]
seller_3=[]
sat_4=[]
sat_count2=[]

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
        
count = 0

reple_result = soup.find('div', class_='thumbnail_list').find('ul',class_='thumbtype_200x200')
slist = reple_result.find_all('li')

#이미지 추출하기
img_file_no = 0

for li in slist:
                
            os.chdir(img_dir)

            try :
                  photo = li.find('div','pub_photo').find('img')['src']
            except AttributeError :
                continue
            
            img_file_no += 1

            urllib.request.urlretrieve(photo,str(img_file_no)+'.jpg')
            time.sleep(1)

            if img_file_no > cnt :
                break


# 텍스트 추출하기
for li in slist:

            
            f = open(ff_name, 'a',encoding='UTF-8')
            f.write("-----------------------------------------------------"+"\n")

            print("-" *70)
            try :
             ranking = li.find('div',class_='product_conts').find('em').get_text()
            except AttributeError :
             ranking = ''
             print(ranking.replace("\n",""))
            else :
             print("1.판매순위:",ranking)

             f.write('1.판매순위:'+ ranking + "\n")
             
            try :
             title = li.find('div', class_='pup_title').get_text()
            except AttributeError :
             title = ''
             print(title.replace("\n",""))
             f.write('2.제품소개:'+ title + "\n")
            else :
             print("2.제품소개:", title.replace("\n",""))

             count += 1
             
             f.write('2.제품소개:'+ title.replace("\n","") + "\n")
            
             
             price = li.find('div', class_='pub_priceW').get_text()
             print("3.가격:", price.replace("\n",""))
             f.write('3.가격:'+ price.replace("\n","") + "\n")
     
             seller = li.find('div',class_='seller_id').get_text()
             seller_2 = seller.split(')')
             print('4.판매자:',seller_2[0].replace("\n",""),')',sep='')
             f.write('4.판매자:'+ seller_2[0].replace("\n","") + "\n")

             try :
                sat = seller_2[1]
             except IndexError :
                sat = '0'
                print('5.만족도:',sat)
                f.write('5.만족도:'+ sat + "\n")
             else :
                sat_2 = sat.split('(')
                sat_3 = sat_2[0].split(':')
                try :
                    print('5.만족도:',sat_3[1].replace("\n",""))
                    f.write('5.만족도:'+ sat_3[1].replace("\n","") + "\n")
                except IndexError :
                    print('5.만족도: 0')
                    f.write('5.만족도: 0' + "\n")
             try :
                sat_count = sat_2[1]
             except IndexError :
                sat_count='0'
                print('6.만족도 리뷰 건수: ',sat_count)
                f.write('6.만족도 리뷰건수:'+ sat_count + "\n")
             else :
                print('6.만족도 리뷰 건수:',sat_count.replace("\n",""))
                f.write('6.만족도 리뷰건수:'+ sat_count + "\n")

                            
             print("-" *70)
             print("\n")
             
             f.close( )
              
             time.sleep(0.5)

             
             
             ranking2.append(ranking)
             title2.append(title.replace("\n",""))
             price2.append(price.replace("\n",""))
             seller_3.append(seller_2[0].replace("\n",""))

             try :
               sat_4.append(sat_3[1].replace("\n",""))
             except IndexError :
               sat_4.append(0)

             try :   
               sat_count2.append(sat_count)
             except IndexError :
               sat_count2.append(0)

             if count == cnt :
                break
                    

#step 9. 엑셀 형태로 저장하기
              
best_seller = pd.DataFrame()

best_seller['판매순위']=ranking2
best_seller['제품소개']=pd.Series(title2)
best_seller['판매가격']=pd.Series(price2)
best_seller['판매자']=pd.Series(seller_3)
best_seller['만족도']=pd.Series(sat_4)
best_seller['만족도 건수']=pd.Series(sat_count2)


# csv 형태로 저장하기
best_seller.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
best_seller.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time


# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

#이미지 삽입하기


excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
      col_name='C'+str(a)
      col_name2.append(col_name)

for b in range(1,cnt+1) :
      file_name=img_dir+'\\'+str(b)+'.jpg'
      file_name2.append(file_name)
      
for i in range(0,cnt) :
      rng = ws.Range(col_name2[i])
      image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
      excel.Visible=True
      excel.ActiveWorkbook.Save()


driver.close( )

연습문제 9-6. 11번가 사이트의 Best Seller 상품 정보 추출하기
   1.크롤링 할 건수는 몇건입니까?(1 - 100 사이값 입력): 20
   2.파일을 저장할 폴더명만 쓰세요(예:c:\temp\):c:\data\


   요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~
----------------------------------------------------------------------
1.판매순위: 1
2.제품소개: [10만원권] 신세계백화점 상품권 11Pay 4,000P 적립
3.가격: 100,000원
4.판매자:기프티콘_11번가(skpgifticon)
5.만족도: 93%
6.만족도 리뷰 건수: 4,549건
----------------------------------------------------------------------


----------------------------------------------------------------------
1.판매순위: 2
2.제품소개: [베베숲] 아기물티슈 네이처골드 60매 캡 10팩
3.가격: 19,900원~
4.판매자:베베숲물티슈(archeint)
5.만족도: 94%
6.만족도 리뷰 건수: 5,572건
----------------------------------------------------------------------


----------------------------------------------------------------------
1.판매순위: 3
2.제품소개: 20%/25% 2종쿠폰! 데싱디바 단독세트 외 페디/매니 신상컬렉션 총집합
3.가격: 10,000원2%세일9,800원~
4.판매자:데싱디바(dashingdiva)
5.만족도: 91%
6.만족도 리뷰 건수: 12,852건
----------------------------------------------------------------------


------------------